## Uludag Sozluk Web Scraping

In [1]:
import requests
from bs4 import BeautifulSoup
from tqdm import tqdm
import json
import string
import time,os
import numpy as np
import pandas as pd
import re

import requests
import time, os

from selenium import webdriver
from selenium.webdriver import Chrome
from selenium.webdriver.common.keys import Keys

In [2]:
url="https://www.uludagsozluk.com/kategori/teknoloji/"
r=requests.get(url)

In [3]:
r.status_code

200

In [4]:
soup=BeautifulSoup(r.content,"lxml")

In [5]:
page_count=len(soup.find_all("div",{"class":"paginationp"})[0].find_all("a"))

In [6]:
page_count

10

In [7]:
comments_list=[]
likes_list=[]
dislike_list=[]
date_list=[]

for page in range(1,page_count+1):
    pageRequest=requests.get(str(url)+"{}/".format(page))
    #print(pageRequest.url)
    pageSoup=BeautifulSoup(pageRequest.content,"lxml")
    entrys=pageSoup.find_all("li",{"class":"rowli"})
    for entry in entrys:
        entryLink="https://www.uludagsozluk.com"+entry.a.get("href")
        entryName=entry.a.get("title")
        entryR=requests.get(entryLink)
        entry_soup=BeautifulSoup(entryR.content,"lxml")
        comment_page_count=len(entry_soup.find("select").find_all("option"))+1
        
        for comment_page in range(1,comment_page_count+1):
            entryR2=requests.get(entryLink+"{}/".format(comment_page))
            entry_soup2=BeautifulSoup(entryR2.content,"lxml")
            comments=entry_soup2.find_all("li",{"class":"li_capsul_entry"})
            
            for comment in comments:
                comments_list.append(comment.find("div",{"class":"entry"}).find("div",{"class":"entry-p"}).text.strip())
                likes_list.append(comment.find_all("a",{"class":"butoy"})[0].text.strip())
                dislike_list.append(comment.find_all("a",{"class":"butoy"})[1].text.strip())
                date_list.append(comment.find("a",{"class":"entry-date"}).text.strip())



In [8]:
len(comments_list),len(likes_list),len(dislike_list),len(date_list)

(5899, 5899, 5899, 5899)

In [9]:
df=pd.DataFrame()
df["Comment"]=comments_list
df["Like"]=likes_list
df["Dislike"]=dislike_list
df["Date"]=date_list

In [11]:
df.to_csv("Uludag_Teknoloji.csv")